In [1]:
import pandas as pd
import xml.etree.ElementTree as elt
import re
import numpy as np

In [2]:
atc_drug_bank = pd.DataFrame()
atc_drug_bank["atc"] = "N/A"
atc_drug_bank["name"] = "N/A"
atc_drug_bank["description"] = "N/A"
atc_drug_bank["code"] = "N/A"
atc_drug_bank["fda_code"] = "N/A"

i = -1
name_first = False
des_first = False

for event, elem in elt.iterparse("../databaze/full database.xml", events=("end",)):
    if elem.tag == "{http://www.drugbank.ca}drugbank-id" and elem.attrib.pop("primary", "nothing") == "true":
        i += 1
        if i % 100 == 0:
            print(i)
            
        name_first = True
        des_first = True
        atc_drug_bank.at[i, "code"] = elem.text
        atc_drug_bank.at[i, "fda_code"] = []
    
    if elem.tag == "{http://www.drugbank.ca}name" and name_first:
        atc_drug_bank.at[i, "name"] = elem.text
        name_first = False

    if elem.tag == "{http://www.drugbank.ca}description" and des_first:
        atc_drug_bank.at[i, "description"] = elem.text
        des_first = False

    if elem.tag == "{http://www.drugbank.ca}atc-codes":
        atc_drug_bank.at[i, "atc"] = [x.attrib["code"] for x in elem.findall("{http://www.drugbank.ca}atc-code")]

    if elem.tag == "{http://www.drugbank.ca}fda-label":
        atc_drug_bank.at[i, "fda_code"] = atc_drug_bank["fda_code"][i] + re.findall("DB[0-9]+", elem.text)

        
# atc_drug_bank.to_csv("databaze/drugbank_to_atc.csv")

0
100
200
300


KeyboardInterrupt: 

In [3]:
atc_drug_bank["fda_code"].apply(lambda x: len(x)).max()

1

In [4]:
atc_drug_bank["fda_code"] = atc_drug_bank["fda_code"].apply(lambda x: x[0] if len(x) >= 1 else np.nan)
atc_drug_bank["fda_code"] = atc_drug_bank.apply(lambda x: np.nan if x.code == x.fda_code else x.fda_code, axis=1)


for i in atc_drug_bank[atc_drug_bank["atc"].isna()].index:
    atc_drug_bank.at[i, "atc"] = []


for i in atc_drug_bank["fda_code"].dropna().index:
    j = atc_drug_bank[atc_drug_bank["code"] == atc_drug_bank["fda_code"][i]].index[0]
    atc_drug_bank.at[j, "atc"] = atc_drug_bank["atc"][i] + atc_drug_bank["atc"][j]


atc_drug_bank.head(10)

,atc,name,description,code,fda_code
0,[B01AE02],Lepirudin,Lepirudin is a recombinant hirudin formed by 6...,DB00001,NaN
1,[L01FE01],Cetuximab,Cetuximab is a recombinant chimeric human/mous...,DB00002,NaN
2,[R05CB13],Dornase alfa,Dornase alfa is a biosynthetic form of human d...,DB00003,NaN
3,[L01XX29],Denileukin diftitox,A recombinant DNA-derived cytotoxic protein co...,DB00004,NaN
4,[L04AB01],Etanercept,Dimeric fusion protein consisting of the extra...,DB00005,NaN
5,[B01AE06],Bivalirudin,Bivalirudin is a synthetic 20 residue peptide ...,DB00006,NaN
6,"[L02AE51, L02AE02]",Leuprolide,Leuprolide is a synthetic 9-residue peptide an...,DB00007,NaN
7,[],Leuprolide acetate,NaN,DBSALT000105,NaN
8,"[L02AE51, L02AE02]",Leuprolide mesylate,Leuprolide 2wk 1 mg/0.2 ml kit,DBSALT003182,DB00007
9,"[L03AB11, L03AB61]",Peginterferon alfa-2a,Peginterferon alfa-2a is a form of recombinant...,DB00008,NaN


In [5]:
atc_drug_bank = atc_drug_bank[atc_drug_bank["fda_code"].isna()]
atc_drug_bank = atc_drug_bank[atc_drug_bank["atc"].apply(lambda x: len(x) != 0)]
atc_drug_bank["atc"] = atc_drug_bank["atc"].apply(lambda x: list(set(x)))
atc_drug_bank = atc_drug_bank.drop(["fda_code"], axis=1)

In [6]:
atc_drug_bank["description"] = atc_drug_bank["description"].fillna("")
atc_drug_bank["description"] = atc_drug_bank.apply(lambda x: "" if "DBSALT" in x.code else x.description, axis=1)

In [7]:
new = pd.DataFrame()
new["atc"] = "N/A"
new["name"] = "N/A"
new["description"] = "N/A"
new["code"] = "N/A"

for i in range(atc_drug_bank["atc"].apply(lambda x: len(x)).max() + 1):
    temp = pd.DataFrame()
    filtered = atc_drug_bank[atc_drug_bank["atc"].apply(lambda x: len(x) > i)].copy()
    temp["atc"] = filtered["atc"].apply(lambda x: x[i])
    temp["name"] = filtered["name"]
    temp["description"] = filtered["description"]
    temp["code"] = filtered["code"]

    temp.index = temp.index + len(new)
    new = pd.concat([new, temp])

atc_drug_bank = new

In [8]:
atc_drug_bank.reset_index(drop=True).to_csv("../databaze/drugbank_to_atc.csv")